# Open in Colab
<a target="_blank" href="https://colab.research.google.com/github/Nicolepcx/ai-agents-the-definitive-guide/blob/main/CH01/ch01_code_examples.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# About this notebook

This notebook is a compact, end to end primer on turning a plain LLM call into a small but real agent that can use tools, keep state across turns, and trace its own reasoning flow.

## What it shows

* **Stateless LLM call** with `langchain-openai` to warm up.
* **Tool use in a loop** with two simple tools

  1. `internet_search` via SerpAPI for fresh information
  2. `calculator` powered by `numexpr` for fast math.
* **A minimal LangGraph agent** with

  * a typed state that accumulates `messages` using `add_messages`
  * an `llm` node bound to tools and a `ToolNode` for execution
  * a router that decides when to call tools or stop
  * in memory checkpoints via `MemorySaver`
  * separate threads to branch conversations and compare memory.

## Why these examples

1. You see the **bare minimum loop** that handles tool calls without any framework state.
2. You then see the **same idea done right** with LangGraph, so you get clean routing, checkpoints, and thread scoped memory.
3. You get **trace utilities** to print node updates and a short state snapshot, which makes debugging and teaching much easier.

## How to run it

* Load keys from `.env` or set them with `%env`. If missing, the notebook will ask interactively.
* Call `gpt-5-mini` or any other LLM you like compatible with OAI API, once for a quick check, then switch to `gpt-4o` bound to tools.
* Run a **two step task**:
  * get the **current air temperature in New York City** using `internet_search`
  * **square the temperature** with `calculator`.
* Repeat the task inside a **LangGraph** app, then branch a **second thread** that converts the temperature to Fahrenheit and reports both.
  You will see per node updates and a final memory snapshot for each thread.

## Key ideas to notice

* **Binding tools to the model** and letting the model choose when to call them.
* A **minimal router** that checks for `tool_calls` and either transitions to the `ToolNode` or ends.
* **Thread ids** for independent memories and reproducible debugging.
* **Deterministic setups** where helpful
  `temperature=0`, explicit `recursion_limit`, and short format prompts to produce consistent outputs.

## Swap and extend

* You can swap `SerpAPIWrapper` with any other search tool that returns text.
* Add your own tools and drop them into `tools` and `tool_map`.
* Replace `MemorySaver` with a persistent checkpointer if you want long lived sessions.
* Add nodes for planning, validation, or guardrails if you want a larger graph.

## Requirements and notes

* You need a working [`OPENAI_API_KEY`](https://platform.openai.com/api-keys) and a [`SerpAPIWrapper`](https://serpapi.com/).
* Internet search results change. The reported temperature and snippets will vary by time and source.
* Tool calls count toward tokens and external API usage. Keep an eye on cost.


# Dependencies

In [ ]:
!pip install -q langgraph==0.6.7 langchain-openai==0.3.33 python-dotenv==1.1.1 langchain_community google-search-results

  Preparing metadata (setup.py) ... done


# Imports for API

In [ ]:

from dotenv import load_dotenv
import os

# Imports

In [ ]:
# Standard library
import os
import math
import json
import numexpr
from typing import List, Dict, Any, TypedDict, Annotated

# LangChain core
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_community.utilities import SerpAPIWrapper

# LangGraph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver


In [ ]:
# --- API Key Setup ---
# Option 1 (preferred): create a `.env` file in your project folder with:
# OPENAI_API_KEY=your_openai_key_here
# SERPAPI_API_KEY=your_serpapi_key_here
#
# Option 2: set it directly in the notebook with magic:
# %env OPENAI_API_KEY=your_openai_key_here
# %env SERPAPI_API_KEY=your_serpapi_key_here

from dotenv import load_dotenv
import os

# Load from .env if available
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
serp_api_key = os.getenv("SERPAPI_API_KEY")

# Fallback: ask if still missing
if not OPENAI_API_KEY:
    print("⚠️ OPENAI_API_KEY not found. You can set it with `%env` in the notebook or enter it below.")
    OPENAI_API_KEY = input("Enter your OPENAI_API_KEY: ").strip()

if not serp_api_key:
    print("⚠️ SERPAPI_API_KEY not found. You can set it with `%env` in the notebook or enter it below.")
    serp_api_key = input("Enter your SERPAPI_API_KEY: ").strip()

print("✅ API keys loaded successfully!")



# First Example: Stateless LLM call with LangChain
____

## Setup LLM

In [ ]:
llm = ChatOpenAI(model="gpt-5-mini")
response = llm.invoke("What are AI agents?")
print(response.content)


Short answer
An AI agent is a software (or embodied) system that perceives its environment, makes decisions, and takes actions to achieve goals — usually with some degree of autonomy and adaptability.

Key characteristics
- Perception: senses inputs (camera, mic, sensors, API data, user text).
- Decision-making: chooses actions based on goals, models, or learned policies.
- Action: affects the environment (move a robot, send a message, place a trade).
- Autonomy: operates without requiring step-by-step human control.
- Goal-directedness: usually tries to maximize a reward or satisfy objectives.

Common types and examples
- Reactive agents: map inputs directly to actions (simple controllers, thermostats).
- Deliberative/planning agents: build internal models and plan ahead (robot path planners).
- Learning agents: improve with experience (reinforcement learning agents, recommendation systems).
- Hybrid agents: combine rules, planning, and learning.
- Embodied agents: robots, autonomous 

## Defining tools for a stateless run


## Tools

In [ ]:
@tool("internet_search")
def internet_search(query: str) -> str:
    """Search Google via SerpAPI for up to date information."""
    serp_api_key = os.environ["SERPAPI_API_KEY"]
    params = {"engine": "google", "gl": "us", "hl": "en"}
    search = SerpAPIWrapper(params=params, serpapi_api_key=serp_api_key)
    return search.run(query)

@tool("calculator")
def calculator(expression: str) -> str:
    """Evaluate a single line mathematical expression with numexpr."""
    local_dict = {"pi": math.pi, "e": math.e}
    out = numexpr.evaluate(
        expression.strip(),
        global_dict={},
        local_dict=local_dict,
    )
    return str(out)

tools = [internet_search, calculator]

tool_map: Dict[str, Any] = {t.name: t for t in tools}

## Model bound to tool

In [ ]:
llm = ChatOpenAI(model="gpt-4o").bind_tools(tools, tool_choice="any")

## Stateless single run with tool loop

In [ ]:
def run_once(prompt: str, max_steps: int = 4) -> str:
    messages = [HumanMessage(content=prompt)]
    for _ in range(max_steps):
        ai: AIMessage = llm.invoke(messages)
        messages.append(ai)

        calls = getattr(ai, "tool_calls", None) or []
        if not calls:
            break

        for call in calls:
            name = call["name"]
            args = call.get("args", {})
            result = tool_map[name].invoke(args)
            messages.append(ToolMessage(
                content=str(result),
                name=name,
                tool_call_id=call["id"]
            ))
    return messages[-1].content

print(run_once("""Two step task.

Step 1: Use internet_search to get the current air temperature in New York City today. Show the exact query you used, the top source title and snippet, and extract a numeric temperature in Celsius. Return this temperature as feedback for Step 2.

Step 2: Using the Celsius value from Step 1, compute its square with calculator. Show the exact expression you used and the numeric result.

Important: Give a short final answer in this format:
Current temperature:
Square of current temperature:"""))



256


# Tools

In [ ]:
@tool("internet_search")
def internet_search(query: str) -> str:
    """Search Google via SerpAPI for up to date information."""
    serp_api_key = os.environ["SERPAPI_API_KEY"]
    params = {"engine": "google", "gl": "us", "hl": "en"}
    search = SerpAPIWrapper(params=params, serpapi_api_key=serp_api_key)
    return search.run(query)

@tool("calculator")
def calculator(expression: str) -> str:
    """Evaluate a single line mathematical expression with numexpr."""
    local_dict = {"pi": math.pi, "e": math.e}
    out = numexpr.evaluate(
        expression.strip(),
        global_dict={},
        local_dict=local_dict,
    )
    return str(out)

tools = [internet_search, calculator]
tool_map: Dict[str, Any] = {t.name: t for t in tools}


# Model bound to tools

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=800).bind_tools(tools, tool_choice="auto")

# Minimal tool loop (stateless)

In [ ]:

def run_once(prompt: str, max_steps: int = 8) -> str:
    messages: List[HumanMessage | AIMessage | ToolMessage] = [HumanMessage(content=prompt)]
    last_ai: AIMessage | None = None

    for _ in range(max_steps):
        ai: AIMessage = llm.invoke(messages)
        messages.append(ai)
        last_ai = ai

        calls = getattr(ai, "tool_calls", None) or []
        if not calls:
            # Model produced a final answer
            return messages[-1].content

        # Execute tool calls and feed observations back
        for call in calls:
            name = call["name"]
            args = call.get("args", {}) or {}
            result = tool_map[name].invoke(args)
            messages.append(ToolMessage(
                content=str(result),
                name=name,
                tool_call_id=call.get("id")
            ))

    # If we exit the loop without a clean final AI message, force a wrap up
    messages.append(HumanMessage(content="""
Finish now. Give a short final answer in this exact format:

Current temperature:
Square of current temperature:
""".strip()))
    final_ai: AIMessage = llm.invoke(messages)
    return final_ai.content

print(run_once("""Two step task.

Step 1: Use internet_search to get the current air temperature in New York City today. Show the exact query you used, the top source title and snippet, and extract a numeric temperature in Celsius. Return this temperature as feedback for Step 2.

Step 2: Using the Celsius value from Step 1, compute its square with calculator. Show the exact expression you used and the numeric result.

Important: Give a short final answer in this format:
Current temperature:
Square of current temperature:"""))


Current temperature: 16°C
Square of current temperature: 256


## Tools

In [ ]:
@tool("internet_search")
def internet_search(query: str) -> str:
    """Search Google via SerpAPI for up to date information."""
    serp_api_key = os.environ["SERPAPI_API_KEY"]
    params = {"engine": "google", "gl": "us", "hl": "en"}
    search = SerpAPIWrapper(params=params, serpapi_api_key=serp_api_key)
    return search.run(query)

@tool("calculator")
def calculator(expression: str) -> str:
    """Evaluate a single line mathematical expression with numexpr."""
    local_dict = {"pi": math.pi, "e": math.e}
    out = numexpr.evaluate(
        expression.strip(),
        global_dict={},
        local_dict=local_dict,
    )
    return str(out)

tools = [internet_search, calculator]


## Build a minimal LangGraph with state, nodes, and routing

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=800).bind_tools(tools)

def llm_node(state: AgentState) -> AgentState:
    ai = llm.invoke(state["messages"])
    return {"messages": [ai]}

tool_node = ToolNode(tools=tools)

graph = StateGraph(AgentState)
graph.add_node("llm", llm_node)
graph.add_node("tools", tool_node)
graph.add_edge(START, "llm")

def route(state: AgentState):
    last = state["messages"][-1]
    calls = getattr(last, "tool_calls", None) or []
    return "tools" if calls else END

graph.add_conditional_edges("llm", route, {"tools": "tools", END: END})
graph.add_edge("tools", "llm")

## Compile with in-memory checkpoints and configure a thread

In [ ]:
checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)


## Trace execution and inspect state and memory

In [ ]:
def _short(msg: BaseMessage, max_len: int = 140) -> str:
    """Compact one-line view of a message."""
    role = type(msg).__name__.replace("Message", "").lower()
    content = getattr(msg, "content", "")
    if isinstance(content, list):
        # some tool outputs can be list payloads
        try:
            content = json.dumps(content)
        except Exception:
            content = str(content)
    text = str(content).replace("\n", " ").strip()
    if len(text) > max_len:
        text = text[: max_len - 3] + "..."
    # include tool name or function call info when available
    if hasattr(msg, "tool_calls") and getattr(msg, "tool_calls"):
        tnames = [tc.get("name", "tool") for tc in msg.tool_calls]
        return f"{role}: tool_calls -> {tnames}"
    if isinstance(msg, ToolMessage):
        return f"{role}({msg.name}): {text}"
    return f"{role}: {text}"

## Trace execution and inspect state and memory

In [ ]:
def print_state_snapshot(app, config, title: str):
    """Print current graph state and memory for a given thread."""
    snap = app.get_state(config)
    values = snap.values or {}
    msgs: List[BaseMessage] = values.get("messages", [])
    print(f"\n=== {title} | state snapshot ===")
    print(f"messages: {len(msgs)} total")
    for i, m in enumerate(msgs[-5:], start=max(0, len(msgs)-5) + 1):
        print(f"  {i:>3}: {_short(m)}")
    # show routing info and queued tasks if present
    nxt = getattr(snap, "next", None)
    tasks = getattr(snap, "tasks", None)
    if nxt:
        print(f"next nodes: {list(nxt)}")
    if tasks:
        print(f"queued tasks: {tasks}")
    # minimal memory view via checkpointer for this thread
    # MemorySaver keeps one latest checkpoint per thread by default, so show existence
    print("memory: in-memory checkpoint present for this thread")

## Trace execution and inspect state and memory

In [ ]:
def run_with_tracing(app, input_state: AgentState, config, title: str):
    """Run the graph while printing per-node updates and final memory."""
    print(f"\n=== {title} | execution trace ===")
    final = None
    # stream_mode="updates" surfaces node-level updates
    for event in app.stream(input_state, config=config, stream_mode="updates"):
        for node, upd in event.items():
            # upd is a dict like {"messages": [<new msg>]} or tool results
            keys = list(upd.keys())
            print(f"[enter {node}] updated: {keys}")
            # if messages updated, print the last one briefly
            msgs = upd.get("messages") or []
            if msgs:
                print(f"  {_short(msgs[-1])}")
            print(f"[leave {node}]")
            final = upd
    # show final assistant message from app.get_state
    print_state_snapshot(app, config, title=f"{title} | after run")
    snap = app.get_state(config)
    msgs = snap.values.get("messages", [])
    return msgs[-1].content if msgs else ""

In [ ]:
# configs
cfg = {"configurable": {"thread_id": "nyc-weather-session"}}

## Turn 1: get the current NYC air temperature in Celsius

In [ ]:
turn1_answer = run_with_tracing(
    app,
    {"messages": [HumanMessage(content="Get the current air temperature in New York City in Celsius.")]},
    config={**cfg, "recursion_limit": 20},
    title="TURN 1",
)
print("\nTURN 1 (final assistant):\n", turn1_answer)

## Turn 2: square that temperature in the same thread

In [ ]:
turn2_answer = run_with_tracing(
    app,
    {"messages": [HumanMessage(content="Now compute the square of that temperature.")]},
    config={**cfg, "recursion_limit": 20},
    title="TURN 2",
)
print("\nTURN 2 (final assistant):\n", turn2_answer)


## Branch a parallel thread for a different follow-up

In [ ]:
cfg_branch = {"configurable": {"thread_id": "nyc-weather-session-branch"}}
branch_answer = run_with_tracing(
    app,
    {"messages": [HumanMessage(content="Instead of squaring, convert it to Fahrenheit and report both.")]},
    config=cfg_branch,
    title="BRANCH",
)
print("\nBRANCH (final assistant):\n", branch_answer)


In [ ]:



# show consolidated memory views for both threads
print_state_snapshot(app, cfg, title="MAIN THREAD memory view")
print_state_snapshot(app, cfg_branch, title="BRANCH THREAD memory view")



=== TURN 1 | execution trace ===
[enter llm] updated: ['messages']
  ai: tool_calls -> ['internet_search']
[leave llm]
[enter tools] updated: ['messages']
  tool(internet_search): {'type': 'weather_result', 'temperature': '18', 'unit': 'Celsius', 'precipitation': '0%', 'humidity': '80%', 'wind': '23 km/h', 'location...
[leave tools]
[enter llm] updated: ['messages']
  ai: The current air temperature in New York City is 18°C.
[leave llm]

=== TURN 1 | after run | state snapshot ===
messages: 4 total
    1: human: Get the current air temperature in New York City in Celsius.
    2: ai: tool_calls -> ['internet_search']
    3: tool(internet_search): {'type': 'weather_result', 'temperature': '18', 'unit': 'Celsius', 'precipitation': '0%', 'humidity': '80%', 'wind': '23 km/h', 'location...
    4: ai: The current air temperature in New York City is 18°C.
memory: in-memory checkpoint present for this thread

TURN 1 (final assistant):
 The current air temperature in New York City is 18°C.

===